Importing cellular automata & optimization classes, and other stuff

In [1]:
import os
import sys
import shutil

from typing import List, Type, Callable, Dict
from numpy import int32
from numpy._typing import NDArray
import importlib

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))

#from cax_sica.blender import Lattice, clear_initial
from cax_sica.genetic import Optimizer, Mutator, RulesetMutator, ArbitraryRulesetMutator, MutationSet
from cax_sica.objectives import surface_to_vol

import numpy as np
import pandas as pd

import time

Setting up optimizer and data logging code

In [2]:
def log_mutation(data_list: List[Dict], mutations: List[MutationSet], objective_val: float):
    """
    Given the data list reference, the mutation set, and the objective value after applying it, add it to the data logging list
    """
    ic_cell_pos = []
    ic_state_old = []
    ic_state_new = []
    srt_cell_pos = []
    srt_state_old = []
    srt_state_new = []

    """
    Important difference from original genetic algorithm: 
    ic_mut is a 4 membered list showing the mutated position in the IC.
    Example: [0,0,0,1], meaning that at IC pos (0,0) the state 0 is modified to be 1.
    srt_mut is a 6 membered list showing the mutated position in the ruleset.
    Example: [0,0,0,0,0,1], meaning that at SRT rule pos (0,0,0) index 0 the rule 0 is modified to be 1.
    For a 2 state SRT there are 18 different cells for mutation: 0 - 17.
    """
    for ic_mut in mutations.ic_mutations:
        ic_cell_pos.append(tuple(ic_mut[0:2]))
        ic_state_old.append(ic_mut[2])
        ic_state_new.append(ic_mut[3])
    for srt_mut in mutations.srt_mutations:
        srt_cell_pos.append(tuple(srt_mut[0:4]))
        srt_state_old.append(srt_mut[4])
        srt_state_new.append(srt_mut[5])
    
    data_list.append({
        "ic_cell_pos": np.array(ic_cell_pos), 
        "ic_state_old": np.array(ic_state_old), 
        "ic_state_new": np.array(ic_state_new), 
        "srt_cell_pos": np.array(srt_cell_pos), 
        "srt_state_old": np.array(srt_state_old), 
        "srt_state_new": np.array(srt_state_new), 
        "objective": objective_val,
    })

def run_experiment(iters: int, grid_sz: int, ruleset_mutator_class: Type[Mutator], rule_set: List[NDArray], opt_func: Callable[[NDArray[int32]], int],
                   srt_num_mutate: int, ic_num_mutate: int, rule_mutate_prob: float, strict: bool = False, num_strict: bool = True, ic_enable: bool = True, srt_enable: bool = True):
    """
    Runs an experiment with the below hyperparameters:

    :param iters: The number of iterations the mutation algorithm (updating both IC and SRT) is going to run for
    :param grid_sz: The size of the square grid that we're going to update each iteration
    :param ruleset_mutator_class: The type of mutator used: RulesetMutator or ArbitraryRulesetMutator
    :param rule_set: The list of possible rulesets if RulesetMutator is used
    :param opt_func: The functions that gives the performance metric we're going to optimize
    :param srt_num_mutate: The number of SRT cells for which we're going to mutate the rule applied, each iteration
    :param ic_num_mutate: The number of IC cells for which we're going to mutate the rule applied, each iteration
    :param rule_mutate_prob: The probability, for each neighbor state tensor of the rule of a cell that's selected to be mutated, the final state is mutated
    :param strict: Whether SRT mutations are chosen by cell then rule or by rule directly; for more info, see mutation.py
    :param num_strict: Whether the number of SRT/IC cells mutated will be constant per iteration or variable; for more info, see mutation.py; incompatible with RulesetMutator
    :param ic_enable: Whether the IC will be mutated
    :param srt_enable: Whether the SRT will be mutated
    """
    # RESOLVED: separate SRT and IC mutations to have a certain number of each
    # RESOLVED: add a flag to enable doing only SRT or only IC mutations in an iteration (in optimizer step, and then propagate into mutator)
    ruleset_mutator = ruleset_mutator_class(rules=rule_set, grid_size=grid_sz, mutate_p=1/(grid_sz**2) * (srt_num_mutate+ic_num_mutate), rule_mutate_p=rule_mutate_prob, strict=strict, num_strict = num_strict, ic_ct = ic_num_mutate, srt_ct=srt_num_mutate, ic_enable=ic_enable, srt_enable=srt_enable)

    optim = Optimizer(mutator=ruleset_mutator, objective=lambda grid: opt_func(grid))
    """
    Pandas Dataframe used to log experiment data is:

    ic_cell_pos (np.array) | ic_state_old (np.array) | ic_state_new (np.array) | srt_cell_pos (np.array) | srt_state_old (np.array) | srt_state_new (np.array) | objective (float)
    
    etc.

    initial state for IC is in entry 0 in ic_state_old, and SRT is in entry 0 in srt_state_old

    ic and srt mutation cell positions and states can have an extra dimension in the beginning to indicate they are batch updates
    """

    init_state = optim.state
    
    data_list = [{"ic_cell_pos": grid_sz, 
                  "ic_state_old": init_state.initial, 
                  "ic_state_new": None, 
                  "srt_cell_pos": -1, 
                  "srt_state_old": init_state.rules, 
                  "srt_state_new": None, 
                  "objective": 0}]

    for it in range(iters):
        # if (it%20 == 0):
        #     print(f"Iteration {it}")
        # print(f"On iteration {it+1}...")
        accepted, new, old, mutations = optim.step()
        # data logging
        log_mutation(data_list, mutations, optim.objvalue)
        # if accepted:
        #     print("Got a better state!", optim.objvalue)

    # print(data_list)
    df = pd.DataFrame(data_list)
    # print(df)
    return df

timelogs = []

def repeat_experiment(experiment_name: str, num_expers: int, *args):
    """
    Perform (sequentially) multiple experiments that return a Pandas DataFrame and save all the data

    :param experiment_name: The name of the experiment to save the file
    :param num_expers: Number of times to run the experiment (and save all the data in one file)
    :param *args: The arguments to be passed to the experiment function
    """
    for i in range(num_expers):
        init = time.time()
        print(f'REPETITION {i}')
        ret_data = run_experiment(*args)
        timelogs.append(time.time() - init)
        print(f"Finished rep {i} in {time.time() - init}s")
        ret_data.to_hdf(f'test/{experiment_name}_{i}.h5', key='data', mode='a')


Setting up experiments and gathering data

In [3]:
ITERATIONS_SET = [100]
GRID_SIZE_SET = [32]
NUM_REPEAT = 100
EXPERIMENT_NAME = "default"

for iters in ITERATIONS_SET:
    for grid_sz in GRID_SIZE_SET:
        print(f"RUNNING EXPERIMENT {EXPERIMENT_NAME} WITH {iters} ITERATIONS AND {grid_sz} SIZE GRID")
        #Total param is set default to false meaning that the probability is not the total probability; 
        #so that the program aligns closer to that of the original genetic algorithm
        
        #def probability for Strict Mode = 2/3; def probability for Non-Strict Mode = 5/384
        repeat_experiment(f"{EXPERIMENT_NAME}_{iters}ITERS_{grid_sz}GRID", NUM_REPEAT, iters, grid_sz, ArbitraryRulesetMutator, [[0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0],[0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]], surface_to_vol, 20, 240, 2/3, True, True, True, True)

RUNNING EXPERIMENT default WITH 100 ITERATIONS AND 32 SIZE GRID
REPETITION 0
Finished rep 0 in 4.1591126918792725s


C:\Users\LAMM\AppData\Local\Temp\ipykernel_4344\3823172413.py:115: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['ic_cell_pos', 'ic_state_old', 'ic_state_new', 'srt_cell_pos',
       'srt_state_old', 'srt_state_new', 'objective'],
      dtype='object')]

  ret_data.to_hdf(f'test/{experiment_name}_{i}.h5', key='data', mode='a')


REPETITION 1
Finished rep 1 in 2.0232481956481934s
REPETITION 2


C:\Users\LAMM\AppData\Local\Temp\ipykernel_4344\3823172413.py:115: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['ic_cell_pos', 'ic_state_old', 'ic_state_new', 'srt_cell_pos',
       'srt_state_old', 'srt_state_new', 'objective'],
      dtype='object')]

  ret_data.to_hdf(f'test/{experiment_name}_{i}.h5', key='data', mode='a')


Finished rep 2 in 2.0179736614227295s
REPETITION 3


...

C:\Users\LAMM\AppData\Local\Temp\ipykernel_4344\3823172413.py:115: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['ic_cell_pos', 'ic_state_old', 'ic_state_new', 'srt_cell_pos',
       'srt_state_old', 'srt_state_new', 'objective'],
      dtype='object')]

  ret_data.to_hdf(f'test/{experiment_name}_{i}.h5', key='data', mode='a')


Finished rep 98 in 2.015925884246826s
REPETITION 99


C:\Users\LAMM\AppData\Local\Temp\ipykernel_4344\3823172413.py:115: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['ic_cell_pos', 'ic_state_old', 'ic_state_new', 'srt_cell_pos',
       'srt_state_old', 'srt_state_new', 'objective'],
      dtype='object')]

  ret_data.to_hdf(f'test/{experiment_name}_{i}.h5', key='data', mode='a')


Finished rep 99 in 2.01271390914917s


C:\Users\LAMM\AppData\Local\Temp\ipykernel_4344\3823172413.py:115: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['ic_cell_pos', 'ic_state_old', 'ic_state_new', 'srt_cell_pos',
       'srt_state_old', 'srt_state_new', 'objective'],
      dtype='object')]

  ret_data.to_hdf(f'test/{experiment_name}_{i}.h5', key='data', mode='a')


In [4]:
print(timelogs)

[4.1591126918792725, 2.0232481956481934, 2.0179736614227295, 2.016418218612671, 2.0068745613098145, 2.005204439163208, 2.0039072036743164, 2.0070223808288574, 2.0070748329162598, 2.012687921524048, 2.0069832801818848, 2.0049691200256348, 2.009951114654541, 2.016258955001831, 2.0147340297698975, 2.026940107345581, 2.014636278152466, 2.0030288696289062, 2.004253387451172, 2.000175714492798, 2.001710891723633, 2.019439935684204, 2.0060133934020996, 2.0045361518859863, 2.000542640686035, 2.0064187049865723, 2.003178358078003, 2.0072779655456543, 2.0063116550445557, 2.009917736053467, 2.011671543121338, 2.0009963512420654, 2.0030477046966553, 2.0083625316619873, 2.0099639892578125, 2.0067028999328613, 2.010349988937378, 2.0045671463012695, 2.004502296447754, 2.0109286308288574, 2.0184714794158936, 2.006965398788452, 2.0053584575653076, 2.0137789249420166, 1.9970934391021729, 2.013122081756592, 2.0019123554229736, 2.0040626525878906, 2.009829521179199, 2.0036849975585938, 2.0060770511627197,